# Convex Quantile Regression (CQR)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : June 12, 2020

References:

[1] Kuosmanen, T., Zhou, X. and Dai, S. (2020). How much climate policy has cost for OECD countries?, World Development, 125, p. 104681.

[2] Dai, S., Zhou, X. and Kuosmanen, T. (2020). Forward-looking assessment of the GHG abatement cost: Application to China, Energy Economics, 88, p. 104758.

In [1]:
from pystoned import CQER
import pandas as pd
import numpy as np

In [2]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)
df.head(5)

OPEX  CAPEX  TOTEX  Energy  Length  Customers  PerUndGr
0   681    729   1612      75     878       4933      0.11
1   559    673   1659      62     964       6149      0.21
2   836    851   1708      78     676       6098      0.75
3  7559   8384  18918     683   12522      55226      0.13
4   424    562   1167      27     697       1670      0.03

In [3]:
# output
y  = df['TOTEX']

# inputs
x1  = df['Energy']
x1  = np.asmatrix(x1).T
x2  = df['Length']
x2  = np.asmatrix(x2).T
x3  = df['Customers']
x3  = np.asmatrix(x3).T
x   = np.concatenate((x1, x2, x3), axis=1)

In [4]:
# calculate the quantile model
cet   = "mult"
fun   = "cost"
rts   = "crs"
tau   = 0.9

model = CQER.cqr(y, x, tau, cet, fun, rts)

from pyomo.environ import SolverManagerFactory
solver_manager = SolverManagerFactory('neos')
results = solver_manager.solve(model, opt='knitro', tee=True)

Note: when estimating the multiplicative model, we can choose the solver such as `knitro` or `minos`. When calculating the additive model, we can choose the solver such as `cplex` or `mosek`.

In [5]:
# display estiamtes
model.em.display()
model.ep.display() 
model.b.display() 

em : error term minus
    Size=89, Index=i
    Key : Lower : Value                  : Upper : Fixed : Stale : Domain
      0 :   0.0 :    0.15369340499475542 :  None : False : False :  Reals
      1 :   0.0 :    0.11796949614874495 :  None : False : False :  Reals
      2 :   0.0 :  5.740781480435187e-07 :  None : False : False :  Reals
      3 :   0.0 :     0.1901232411076666 :  None : False : False :  Reals
      4 :   0.0 :  0.0011831641491704009 :  None : False : False :  Reals
      5 :   0.0 :     0.3354871137681057 :  None : False : False :  Reals
      6 :   0.0 :     0.1928017112379842 :  None : False : False :  Reals
      7 :   0.0 :    0.06833583793305759 :  None : False : False :  Reals
      8 :   0.0 :  1.538814639157656e-06 :  None : False : False :  Reals
      9 :   0.0 :      0.297943168710589 :  None : False : False :  Reals
     10 :   0.0 :     0.2058089185297454 :  None : False : False :  Reals
     11 :   0.0 :    0.26564405916943634 :  None : False : False :  R

In [6]:
# retrive the residuals (eminos)
val = list(model.em[:].value)
eminos = np.asarray(val)
eminos

array([1.53693405e-01, 1.17969496e-01, 5.74078148e-07, 1.90123241e-01,
       1.18316415e-03, 3.35487114e-01, 1.92801711e-01, 6.83358379e-02,
       1.53881464e-06, 2.97943169e-01, 2.05808919e-01, 2.65644059e-01,
       8.07303133e-07, 5.74061680e-07, 1.59154609e-01, 3.42843421e-01,
       5.74104547e-07, 1.98101001e-02, 1.75131277e-01, 2.53256886e-01,
       3.24950653e-01, 4.65770721e-01, 5.87187544e-03, 2.36172093e-01,
       6.75356864e-02, 1.21178435e-01, 5.74062400e-07, 3.66196167e-01,
       3.05019064e-01, 1.06916713e-01, 1.87621378e-01, 8.56264093e-01,
       2.73962705e-01, 1.39459457e-06, 1.80668854e-01, 2.23276428e-02,
       3.96928603e-01, 3.42256023e-01, 3.02787654e-01, 1.79835207e-01,
       1.09848464e-01, 1.30697979e-01, 1.15608067e-01, 2.48029760e-01,
       2.81698459e-01, 2.94963982e-01, 1.32601084e-01, 8.43097150e-02,
       3.17978193e-01, 3.64116811e-01, 5.74098114e-07, 2.10824037e-01,
       4.00230719e-02, 1.84821566e-03, 1.45661527e-01, 4.13131757e-01,
      

In [7]:
# retrive the residuals (eplus)
val = list(model.ep[:].value)
eplus = np.asarray(val)
eplus

array([5.74058646e-07, 5.74059213e-07, 1.51118452e-02, 5.74058307e-07,
       5.75287329e-07, 5.74057664e-07, 5.74058292e-07, 5.74060824e-07,
       9.15199002e-07, 5.74057773e-07, 5.74058215e-07, 5.74057877e-07,
       1.97465621e-06, 6.09814137e-02, 5.74058622e-07, 5.74057642e-07,
       6.86142251e-03, 5.74067409e-07, 5.74058441e-07, 5.74057933e-07,
       5.74057678e-07, 5.74057420e-07, 5.73000912e-07, 5.74058033e-07,
       5.74060864e-07, 5.74059128e-07, 5.49309434e-02, 5.74057590e-07,
       5.74057751e-07, 5.74059532e-07, 5.74058320e-07, 5.74057133e-07,
       5.74057860e-07, 9.75270920e-07, 5.74058383e-07, 5.74067645e-07,
       5.74057526e-07, 5.74057638e-07, 5.74057747e-07, 5.74058407e-07,
       5.74059467e-07, 5.74059022e-07, 5.74059242e-07, 5.74057953e-07,
       5.74057821e-07, 5.74057793e-07, 5.74058948e-07, 5.74060252e-07,
       5.74057702e-07, 5.74057595e-07, 7.97553890e-03, 5.74058152e-07,
       5.74064073e-07, 5.74080424e-07, 5.74058741e-07, 5.74057500e-07,
      

In [8]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

import pandas as pd
beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2', 'b3']

beta

b1        b2        b3
Name                               
0.0   10.032550  1.247919  0.006423
1.0    1.053827  1.163357  0.110571
2.0   10.689815  1.127930  0.014118
3.0    9.469924  1.288027  0.005119
4.0    9.341189  1.314425  0.000009
...         ...       ...       ...
84.0  10.096925  1.251932  0.004666
85.0  10.000111  1.242987  0.007778
86.0  10.014573  1.242901  0.007588
87.0   9.157092  1.286918  0.009223
88.0  11.239241  0.964840  0.018219

[89 rows x 3 columns]